In [ ]:
# ======================================================
# RAG LOCAL COMPLET AVEC OLLAMA + POSTGRESQL + PGVECTOR
# Version finale clean (projet académique)
# ======================================================

# ============================
# IMPORTATIONS
# ============================
import psycopg
from psycopg import Cursor
import ollama
import chardet

# ============================
# CONFIGURATION
# ============================
DB_CONNECTION_STR = "dbname=rag_chatbot user=postgres password=summer2025 host=localhost port=5432"
CONVERSATION_FILES = [
    "../data/017_00000012.txt",
    "../data/018_00000013.txt",
    "../data/019_00000014.txt",
    "../data/020_00000015.txt",
    "../data/022_00000017.txt"
]

EMBEDDING_MODEL = "nomic-embed-text"   # 768 dimensions
LLM_MODEL = "llama3.2"                  # modèle génératif

# ============================
# 1. CHARGEMENT ET NETTOYAGE DU TEXTE
# ============================

def load_conversation(file_path: str) -> list[str]:
    """
    Charge le fichier texte, détecte l'encodage
    et retourne une liste de lignes nettoyées.
    """
    with open(file_path, "rb") as f:
        raw = f.read()
        encoding = chardet.detect(raw)["encoding"]
        print("ENCODING DÉTECTÉ =", encoding)

    with open(file_path, "r", encoding=encoding, errors="ignore") as f:
        lines = f.read().split("\n")

    cleaned = [
        line.strip().lstrip(" ")
        for line in lines
        if line.strip() != "" and not line.startswith("<")
    ]
    return cleaned

# ============================
# 2. REGROUPEMENT DU DIALOGUE
# ============================

def group_dialogue(lines: list[str], turns_per_chunk: int = 6) -> list[str]:
    """
    Regroupe les tours de parole pour préserver
    le sens de la conversation.
    """
    chunks = []
    for i in range(0, len(lines), turns_per_chunk):
        chunk = " ".join(lines[i:i + turns_per_chunk])
        chunks.append(chunk)
    return chunks

# ============================
# 3. CHUNKING CONTRÔLÉ (SAFE POUR EMBEDDINGS)
# ============================

def chunk_text(text: str, max_chars: int = 800) -> list[str]:
    """
    Découpe un texte long en sous-chunks
    (≈ 200 tokens) sans dépendre d'un tokenizer spécifique.
    """
    chunks = []
    current = ""

    for sentence in text.split("."):
        if len(current) + len(sentence) < max_chars:
            current += sentence + "."
        else:
            chunks.append(current.strip())
            current = sentence + "."

    if current.strip():
        chunks.append(current.strip())

    return chunks

# ============================
# 4. EMBEDDINGS OLLAMA
# ============================

def calculate_embedding(text: str) -> list[float]:
    response = ollama.embeddings(
        model=EMBEDDING_MODEL,
        prompt=text
    )
    return response["embedding"]

# ============================
# 5. SAUVEGARDE EN BASE
# ============================

def save_embedding(corpus: str, embedding: list[float], cursor: Cursor) -> None:
    cursor.execute(
        "INSERT INTO embeddings (corpus, embedding) VALUES (%s, %s)",
        (corpus, embedding)
    )

# ============================
# 6. RECHERCHE PAR SIMILARITÉ
# ============================

def similar_corpus(query: str, k: int = 3):
    query_embedding = calculate_embedding(query)
    vector_literal = "[" + ",".join(map(str, query_embedding)) + "]"

    with psycopg.connect(DB_CONNECTION_STR) as conn:
        with conn.cursor() as cur:
            cur.execute(
                """
                SELECT corpus, embedding <=> %s::vector AS distance
                FROM embeddings
                ORDER BY distance ASC
                LIMIT %s
                """,
                (vector_literal, k)
            )
            return cur.fetchall()

# ============================
# 7. RAG : GÉNÉRATION DE RÉPONSE
# ============================

def rag_answer(question: str) -> str:
    results = similar_corpus(question)
    context = "\n\n".join([r[0] for r in results])
    prompt = f"""
        Tu es un assistant d'analyse de documents.

            CONTEXTE :{context}
            QUESTION :{question}
            INSTRUCTIONS :
                - Réponds uniquement à partir du contexte
                - Si plusieurs documents sont concernés, synthétise les informations
                - N'ajoute aucune information externe
        """


    response = ollama.chat(
        model=LLM_MODEL,
        messages=[{"role": "user", "content": prompt}]
    )

    return response["message"]["content"]

# ============================
# 8. INGESTION DES DONNÉES
# ============================

with psycopg.connect(DB_CONNECTION_STR) as conn:
    conn.autocommit = True
    with conn.cursor() as cur:
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector")

        cur.execute("DROP TABLE IF EXISTS embeddings")
        cur.execute(
            """
            CREATE TABLE embeddings (
                id SERIAL PRIMARY KEY,
                corpus TEXT NOT NULL,
                embedding VECTOR(768)
            );
            """
        )

        for file_path in CONVERSATION_FILES:
            lines = load_conversation(file_path)
            dialogue_chunks = group_dialogue(lines)

            for dialogue in dialogue_chunks:
                sub_chunks = chunk_text(dialogue)
                for chunk in sub_chunks:
                    emb = calculate_embedding(chunk)
                    save_embedding(chunk, emb, cur)


# ============================
# 9. TEST FINAL
# ============================

if __name__ == "__main__":
    while True:
        question = input("\nPose ta question (ou 'exit' pour quitter) : ")
        if question.lower() in ["exit", "quit"]:
            break
        print("\nRÉPONSE :")
        print(rag_answer(question))



ENCODING DÉTECTÉ = ISO-8859-1
ENCODING DÉTECTÉ = ISO-8859-1
ENCODING DÉTECTÉ = ISO-8859-1
ENCODING DÉTECTÉ = ISO-8859-1
ENCODING DÉTECTÉ = ISO-8859-1
ENCODING DÉTECTÉ = ISO-8859-1
ENCODING DÉTECTÉ = ISO-8859-1
ENCODING DÉTECTÉ = ISO-8859-1
ENCODING DÉTECTÉ = ISO-8859-1
ENCODING DÉTECTÉ = ISO-8859-1

RÉPONSE :
Voici un résumé des conversations :

Conversation 1:

- La personne appelle un assistant pour valider ou attendre les résultats de l'I U T.
- Elle explique qu'elle avait déjà validé, mais ne voyait pas le résultat sur l'écran.
- L'assistant lui propose de regarder après.

Conversation 2 (sujets abordés) :

- La personne dit avoir un sujet à discuter demain matin.
- Elle explique qu'elle a peu de temps et doit appeler rapidement.
- L'assistant lui demande ce que le sujet est.
- La personne explique qu'il n'est pas là, mais elle veut prévenir quelqu'un d'autre.
